In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fahdseddik/arabic-company-reviews")

Using Colab cache for faster access to the 'arabic-company-reviews' dataset.


In [ ]:
!mkdir Data
!cp {path} -r Data

In [ ]:
import pandas as pd
df = pd.read_csv("/content/Data/arabic-company-reviews/CompanyReviews.csv",index_col=0)
df = df.dropna()  # Drops rows with *any* NaN by default
df.head()

,review_description,rating,company
0,رائع,1,talbat
1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,1,talbat
2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,-1,talbat
3,لماذا لا يمكننا طلب من ماكدونالدز؟,-1,talbat
4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,-1,talbat


In [ ]:
# Show imbalance: Value counts with percentages
print("Class Distribution (Counts):")
counts = df['rating'].value_counts().sort_index()
print(counts)

print("\nClass Distribution (Percentages):")
percentages = df['rating'].value_counts(normalize=True).sort_index() * 100
print(percentages.round(2))



Class Distribution (Counts):
rating
-1    14199
 0     1925
 1    23921
Name: count, dtype: int64

Class Distribution (Percentages):
rating
-1    35.46
 0     4.81
 1    59.74
Name: proportion, dtype: float64


In [ ]:
df.sample(5)

,review_description,rating,company
32491,امتهةهةغيتببغرقؤقبناةهىبالاترغىاهةهىتؤبلهبلنا,1,swvl
6914,هو حلو بس ليش وجبة BTS مو حاليه؟؟ ارجو الرد,1,talbat
12963,تمام. وتحربه جميله,1,talbat
20154,يجنن مره طلبت وقالي 60 دقيقه والطلب عندك بس ان...,1,talbat
30146,ليه منطقه حائل مهي موجوده,-1,talbat


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40045 entries, 0 to 132
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review_description  40045 non-null  object
 1   rating              40045 non-null  int64 
 2   company             40045 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [ ]:
# Arabic Review Sentiment Classification using LSTM and Gradio

import pandas as pd
import numpy as np
import re
import pickle
from collections import Counter

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight


# Gradio for UI
import gradio as gr



In [ ]:
class ArabicTextPreprocessor:
    """Comprehensive Arabic text preprocessing"""

    def __init__(self):
        # Arabic stop words (common words to remove)
        self.stop_words = set([
            'في', 'من', 'إلى', 'على', 'هذا', 'هذه', 'ذلك', 'التي', 'الذي',
            'كان', 'كانت', 'لم', 'لن', 'أن', 'إن', 'قد', 'لقد', 'كل', 'بعض',
            'عن', 'عند', 'أو', 'و', 'ف', 'ثم', 'لكن', 'حتى', 'ما', 'لا',
            'نعم', 'كيف', 'متى', 'أين', 'ماذا', 'هل', 'به', 'بها', 'له', 'لها'
        ])

        # Franco-Arabic mapping (common transliterations)
        self.franco_map = {
            '2': 'ا', '3': 'ع', '5': 'خ', '6': 'ط', '7': 'ح', '8': 'ق', '9': 'ص',
            'a': 'ا', 'b': 'ب', 't': 'ت', 'th': 'ث', 'g': 'ج', '7': 'ح',
            'kh': 'خ', 'd': 'د', 'z': 'ذ', 'r': 'ر', 's': 'س', 'sh': 'ش',
            '9': 'ص', 'd': 'ض', 't': 'ط', 'z': 'ظ', '3': 'ع', 'gh': 'غ',
            'f': 'ف', 'q': 'ق', 'k': 'ك', 'l': 'ل', 'm': 'م', 'n': 'ن',
            'h': 'ه', 'w': 'و', 'y': 'ي', 'e': 'ي', 'o': 'و', 'u': 'و'
        }

        # Emoji sentiment mapping
        self.positive_emojis = set(['😊', '😃', '😄', '❤️', '👍', '🌟', '✨', '💯', '🔥', '👏'])
        self.negative_emojis = set(['😢', '😞', '😠', '👎', '💔', '😡', '😤', '🤬'])

    def normalize_arabic(self, text):
        """Normalize Arabic characters"""
        # Replace various forms of alef
        text = re.sub("[إأآا]", "ا", text)
        # Normalize teh marbuta
        text = re.sub("ة", "ه", text)
        # Normalize yeh
        text = re.sub("ى", "ي", text)
        # Remove diacritics (tashkeel)
        text = re.sub(r'[\u064B-\u065F]', '', text)
        # Remove tatweel
        text = re.sub(r'ـ', '', text)
        return text

    def handle_emojis(self, text):
        """Convert emojis to sentiment words"""
        for emoji in self.positive_emojis:
            if emoji in text:
                text = text.replace(emoji, ' رائع ')
        for emoji in self.negative_emojis:
            if emoji in text:
                text = text.replace(emoji, ' سيء ')
        # Remove remaining emojis
        text = re.sub(r'[^\w\s\u0600-\u06FF]', ' ', text)
        return text

    def handle_franco(self, text):
        """Convert Franco-Arabic to Arabic"""
        # Sort by length (longer patterns first)
        for franco, arabic in sorted(self.franco_map.items(), key=lambda x: len(x[0]), reverse=True):
            text = text.replace(franco, arabic)
        return text

    def clean_text(self, text):
        """Main cleaning function"""
        if pd.isna(text):
            return ""

        text = str(text).lower()

        # Handle emojis first
        text = self.handle_emojis(text)

        # Normalize Arabic
        text = self.normalize_arabic(text)

        # Remove English characters (but keep numbers temporarily for franco)
        text = re.sub(r'[a-zA-Z]+', '', text)

        # Remove URLs
        text = re.sub(r'http\S+|www.\S+', '', text)

        # Remove digits
        text = re.sub(r'\d+', '', text)

        # Remove punctuation
        text = re.sub(r'[^\w\s\u0600-\u06FF]', ' ', text)

        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def remove_stop_words(self, text):
        """Remove Arabic stop words"""
        words = text.split()
        filtered = [w for w in words if w not in self.stop_words and len(w) > 1]
        return ' '.join(filtered)

    def preprocess(self, text):
        """Full preprocessing pipeline"""
        text = self.clean_text(text)
        text = self.remove_stop_words(text)
        return text


In [ ]:
# ============================================================================
# 2. LOAD AND PREPROCESS DATA
# ============================================================================

def load_and_preprocess_data(file_path):
    """Load and preprocess the dataset"""
    print("Loading data...")
    df = pd.read_csv(file_path, index_col=0)

    print(f"Dataset shape: {df.shape}")
    print(f"Class distribution:\n{df['rating'].value_counts()}")

    # Initialize preprocessor
    preprocessor = ArabicTextPreprocessor()

    print("\nPreprocessing text...")
    df['cleaned_text'] = df['review_description'].apply(preprocessor.preprocess)

    # Remove empty texts
    df = df[df['cleaned_text'].str.len() > 0].copy()

    # Map ratings to 0, 1, 2 for categorical classification
    df['label'] = df['rating'].map({-1: 0, 0: 1, 1: 2})

    print(f"After cleaning: {df.shape}")

    return df, preprocessor

In [ ]:
# ============================================================================
# 3. TEXT VECTORIZATION & SEQUENCE PREPARATION
# ============================================================================

def prepare_sequences(texts, labels, max_words=10000, max_len=100, tokenizer=None):
    """Convert texts to sequences and pad them"""

    if tokenizer is None:
        # Create and fit tokenizer
        tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
        tokenizer.fit_on_texts(texts)

    # Convert texts to sequences
    sequences = tokenizer.texts_to_sequences(texts)

    # Pad sequences
    padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    # Convert labels to numpy array
    labels = np.array(labels)

    vocab_size = min(len(tokenizer.word_index) + 1, max_words)

    print(f"Vocabulary size: {vocab_size}")
    print(f"Sequence shape: {padded.shape}")

    return padded, labels, tokenizer, vocab_size


In [ ]:
# ============================================================================
# 4. BUILD LSTM MODEL
# ============================================================================

def build_lstm_model(vocab_size, embedding_dim=128, max_len=100, lstm_units=128):
    """Build LSTM model for sentiment classification"""

    model = Sequential([
        # Embedding layer
        Embedding(input_dim=vocab_size,
                 output_dim=embedding_dim,
                 input_length=max_len,
                 mask_zero=True),

        # Spatial dropout to reduce overfitting
        SpatialDropout1D(0.2),

        # Bidirectional LSTM layers
        Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        Bidirectional(LSTM(lstm_units//2, dropout=0.2, recurrent_dropout=0.2)),

        # Dense layers
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),

        # Output layer (3 classes)
        Dense(3, activation='softmax')
    ])

    # Compile model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
def train_model(X_train, y_train, X_val, y_val, vocab_size, max_len=100, epochs=20, batch_size=32, class_weight=None):
    """Train the LSTM model"""

    print("\nBuilding model...")
    model = build_lstm_model(vocab_size, max_len=max_len)

    print(model.summary())

    # Display class weights if provided
    if class_weight:
        print(f"\nUsing class weights: {class_weight}")

    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001, verbose=1)

    print("\nTraining model...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        class_weight=class_weight,  # Accept class weights
        callbacks=[early_stop, reduce_lr],
        verbose=1
    )

    return model, history

In [ ]:
# ============================================================================
# 6. EVALUATION
# ============================================================================

def evaluate_model(model, X_test, y_test):
    """Evaluate model performance"""

    print("\nEvaluating model...")

    # Predictions
    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)

    # Metrics
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Accuracy
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Accuracy: {test_acc:.4f}")

    return y_pred



In [ ]:
# ============================================================================
# 7. PREDICTION FUNCTION
# ============================================================================

class SentimentPredictor:
    """Sentiment prediction wrapper"""

    def __init__(self, model, tokenizer, preprocessor, max_len=100):
        self.model = model
        self.tokenizer = tokenizer
        self.preprocessor = preprocessor
        self.max_len = max_len
        self.label_map = {0: -1, 1: 0, 2: 1}
        self.label_names = {0: 'سلبي (Negative)', 1: 'محايد (Neutral)', 2: 'إيجابي (Positive)'}

    def predict(self, text):
        """Predict sentiment for a single text"""
        # Preprocess
        cleaned = self.preprocessor.preprocess(text)

        if not cleaned:
            return "محايد (Neutral)", 0, [0.0, 1.0, 0.0]

        # Tokenize and pad
        seq = self.tokenizer.texts_to_sequences([cleaned])
        padded = pad_sequences(seq, maxlen=self.max_len, padding='post', truncating='post')

        # Predict
        proba = self.model.predict(padded, verbose=0)[0]
        pred_class = np.argmax(proba)

        # Map to original labels
        sentiment_label = self.label_map[pred_class]
        sentiment_name = self.label_names[pred_class]

        return sentiment_name, sentiment_label, proba.tolist()

In [ ]:
# ============================================================================
# 8. GRADIO INTERFACE
# ============================================================================

def create_gradio_interface(predictor):
    """Create Gradio web interface"""

    def predict_sentiment(text):
        """Prediction function for Gradio"""
        if not text or text.strip() == "":
            return "الرجاء إدخال نص للتحليل", "N/A", ""

        sentiment_name, sentiment_label, probabilities = predictor.predict(text)

        # Format probabilities
        prob_text = f"""
        **احتمالات التصنيف:**
        - سلبي: {probabilities[0]:.2%}
        - محايد: {probabilities[1]:.2%}
        - إيجابي: {probabilities[2]:.2%}
        """

        return sentiment_name, str(sentiment_label), prob_text

    # Create interface
    interface = gr.Interface(
        fn=predict_sentiment,
        inputs=gr.Textbox(
            label="أدخل المراجعة بالعربية",
            placeholder="اكتب مراجعتك هنا...",
            lines=5
        ),
        outputs=[
            gr.Textbox(label="التصنيف"),
            gr.Textbox(label="القيمة العددية"),
            gr.Markdown(label="احتمالات التصنيف")
        ],
        title="🔍 تحليل المشاعر للمراجعات العربية",
        description="نظام ذكي لتحليل المشاعر في المراجعات العربية باستخدام LSTM",
        examples=[
            ["البرنامج رائع جدا وسهل الاستخدام"],
            ["التطبيق سيء ولا يعمل بشكل صحيح"],
            ["البرنامج عادي لا جديد فيه"]
        ],
        theme="soft"
    )

    return interface


In [ ]:
# ============================================================================
# 9. MAIN EXECUTION
# ============================================================================

def main(data_path="/content/Data/arabic-company-reviews/CompanyReviews.csv"):
    """Main execution function"""

    # Configuration
    MAX_WORDS = 5000
    MAX_LEN = 70
    EMBEDDING_DIM = 64
    LSTM_UNITS = 64
    EPOCHS = 3
    BATCH_SIZE = 64

    # Load and preprocess data
    df, preprocessor = load_and_preprocess_data(data_path)

    # Prepare sequences
    X, y, tokenizer, vocab_size = prepare_sequences(
        df['cleaned_text'].values,
        df['label'].values,
        max_words=MAX_WORDS,
        max_len=MAX_LEN
    )

    # Split data
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

    print(f"\nTrain size: {X_train.shape[0]}")
    print(f"Validation size: {X_val.shape[0]}")
    print(f"Test size: {X_test.shape[0]}")





    # Train model
    model, history = train_model(X_train, y_train, X_val, y_val, vocab_size, max_len=MAX_LEN,
                                 epochs=EPOCHS, batch_size=BATCH_SIZE)

    # Evaluate
    evaluate_model(model, X_test, y_test)

    # Save model and tokenizer
    print("\nSaving model and tokenizer...")
    model.save('arabic_sentiment_lstm.h5')
    with open('tokenizer.pickle', 'wb') as f:
        pickle.dump(tokenizer, f)
    with open('preprocessor.pickle', 'wb') as f:
        pickle.dump(preprocessor, f)
    print("Model saved successfully!")

    # Create predictor
    predictor = SentimentPredictor(model, tokenizer, preprocessor, max_len=MAX_LEN)

    # Test predictions
    print("\n" + "="*50)
    print("Testing predictions:")
    print("="*50)
    test_reviews = [
        "البرنامج رائع جدا وسهل الاستخدام",
        "التطبيق سيء ولا يعمل",
        "عادي لا بأس به"
    ]

    for review in test_reviews:
        sentiment, label, probs = predictor.predict(review)
        print(f"\nReview: {review}")
        print(f"Sentiment: {sentiment} (Label: {label})")

    # Launch Gradio interface
    print("\n" + "="*50)
    print("Launching Gradio interface...")
    print("="*50)
    interface = create_gradio_interface(predictor)
    interface.launch(share=True)





In [ ]:
# ============================================================================
# 10. LOADING SAVED MODEL (FOR LATER USE)
# ============================================================================

def load_saved_model():
    """Load previously saved model and launch Gradio"""

    print("Loading saved model...")
    model = load_model('arabic_sentiment_lstm.h5')

    with open('tokenizer.pickle', 'rb') as f:
        tokenizer = pickle.load(f)

    with open('preprocessor.pickle', 'rb') as f:
        preprocessor = pickle.load(f)

    print("Model loaded successfully!")

    # Create predictor
    predictor = SentimentPredictor(model, tokenizer, preprocessor)

    # Launch interface
    interface = create_gradio_interface(predictor)
    interface.launch(share=True)

In [ ]:
# Run the main function
if __name__ == "__main__":
    # Use your existing data path
    main(data_path="/content/Data/arabic-company-reviews/CompanyReviews.csv")

    # To load and use saved model later, uncomment:
    # load_saved_model()

Loading data...
Dataset shape: (40046, 3)
Class distribution:
rating
 1    23921
-1    14200
 0     1925
Name: count, dtype: int64

Preprocessing text...
After cleaning: (38349, 5)
Vocabulary size: 5000
Sequence shape: (38349, 70)

Train size: 26844
Validation size: 5752
Test size: 5753

Building model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_1             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None

Training model...
Epoch 1/3
420/420 ━━━━━━━━━━━━━━━━━━━━ 478s 1s/step - accuracy: 0.7456 - loss: 0.6656 - val_accuracy: 0.8328 - val_loss: 0.4683 - learning_rate: 0.0010
Epoch 2/3
420/420 ━━━━━━━━━━━━━━━━━━━━ 452s 1s/step - accuracy: 0.8566 - loss: 0.4104 - val_accuracy: 0.8326 - val_loss: 0.4499 - learning_rate: 0.0010
Epoch 3/3
420/420 ━━━━━━━━━━━━━━━━━━━━ 450s 1s/step - accuracy: 0.8653 - loss: 0.3781 - val_accuracy: 0.8315 - val_loss: 0.4733 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 2.

Evaluating model...
180/180 ━━━━━━━━━━━━━━━━━━━━ 31s 163ms/step

Classification Report:
              precision    recall  f1-score   support

    Negative       0.81      0.83      0.82      2087
     Neutral       0.00      0.00      0.00       281
    Positive       0.85      0.91      0.88      3385

    accuracy                           0.84      5753
   macro avg       0.55      0.58      0.57      5753
weighted avg       0.79      0.84      0.81   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Test Accuracy: 0.8356

Saving model and tokenizer...
Model saved successfully!

Testing predictions:

Review: البرنامج رائع جدا وسهل الاستخدام
Sentiment: إيجابي (Positive) (Label: 1)

Review: التطبيق سيء ولا يعمل
Sentiment: سلبي (Negative) (Label: -1)

Review: عادي لا بأس به
Sentiment: إيجابي (Positive) (Label: 1)

Launching Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e622b35b59baeced22.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
